# Lab 12: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [ ]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from full_localization import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

In [ ]:
# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

<hr>

# Localization class
The class (defined in [full_localization.py](../full_localization.py)) provides member functions and member variables required for Grid Localization. <br>
Below is a summary of the necessary member functions and variables that will be required to complete this lab:


<table align="left">
    <tr>
        <th style="text-align: left">Member Functions</th>
        <th style="text-align: left">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">init_grid_beliefs()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Initialize the member variable <b>bel</b> with a uniform distribution.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">gaussian(x, mu, sigma)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Returns the relative likelihood of <em>x</em> in a Normal Distribution with mean <em>mu</em> and standard deviation <em>sigma</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">get_observation_data()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Perform the observation loop behavior, where the robot does a 360 degree turn in place while collecting equidistant (in the angular space) sensor data, starting with the first sensor reading taken at the robot's current heading. The number of observations made depends on OBS_PER_CELL defined in the Mapper class. The range measurements are stored in the member variable <em>obs_range_data</em> of class <em>BaseLocalization</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">print_prediction_stats(plot_data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A helper function that displays statistics after your prediction step and plots the ground truth position (green), odometry position (red) and visualizes the prior belief (grayscale grid) if <em>plot_data</em> is True.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">print_update_stats(plot_data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A helper function that displays statistics after your update step and plots the belief position (blue) if <em>plot_data</em> is True.</span></th>
    </tr>
</table>

<table align="left">
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Member Variables</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">obs_range_data</span></th>
        <th style="text-align: left"><span style="font-weight: normal"> A column numpy array containing range measurements made by a robot after completing an observation rotation behavior i.e after calling the function <em>get_observation_data()</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">bel_bar</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A 3D numpy array that is used to represent the prior belief of the robot i.e after the most recent prediction step.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">bel</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A 3D numpy array that is used to represent the belief of the robot i.e after the most recent update step. It is initialized with a uniform distribution at t=0.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">sensor_sigma</span></th>
        <th style="text-align: left"><span style="font-weight: normal">The noise parameter (standard deviation of the Gaussian function) for your sensor model (defined in <a href="../config/world.yaml">world.yaml</a>). </span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">odom_rot_sigma</span></th>
        <th style="text-align: left"><span style="font-weight: normal">The noise parameter (standard deviation of the Gaussian function) for rotation in your odometry motion model (defined in <a href="../config/world.yaml">world.yaml</a>).</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(100,191,100);font-family:monospace">odom_trans_sigma</span></th>
        <th style="text-align: left"><span style="font-weight: normal">The noise parameter (standard deviation of the Gaussian function) for translation in your odometry motion model (defined in <a href="../config/world.yaml">world.yaml</a>).</span></th>
    </tr>
</table>

In [ ]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

# Run the Bayes Filter
The cells below utilizes the functions declared above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In [ ]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

In [ ]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")